In [1]:
import findspark
findspark.init()
import pyspark
import random

In [2]:
import os
os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"]="YES"

In [3]:
from pyspark.sql.functions import *

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("AppName").config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").config("spark.executor.memoryOverhead", "1024").config("spark.driver.memoryOverhead", "1024").config("spark.memory.fraction", "0.9").config("spark.memory.storageFraction", "0.5").config("spark.executor.cores", "2").config("spark.task.maxFailures", "5").config("spark.yarn.am.memory", "4g").getOrCreate()

In [6]:
from pyspark import SparkConf as conf, SparkContext as sc

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions
import time

In [8]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning) 
warnings.filterwarnings('ignore', category=FutureWarning) 
warnings.filterwarnings('ignore', category=UserWarning)

In [9]:
!pip install spacy --quiet
!pip install pyspark --quiet
!pip install langdetect --quiet
!pip install nltk --quiet 
!pip install wordcloud --quiet
!pip install plotly --quiet
!pip install missingno --quiet
!pip install gensim --quiet
!python -m spacy download en_core_web_sm --quiet

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [11]:
!python -m pip install textblob --quiet

In [12]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline
import gensim
from gensim import corpora, models
import matplotlib.pyplot as plt
from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

import matplotlib as mpl
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, when, trim
from pyspark.sql.types import StringType
from langdetect import detect

# Import Libraries
import os
import time
import missingno as msno
import pandas as pd
import numpy as np
import seaborn as sns
import nltk, re, string, collections, unicodedata

from matplotlib import cm, dates
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
from textblob import TextBlob
from langdetect import detect
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

spark = SparkSession.builder.appName("ElectionTweet").getOrCreate()

In [13]:
# def clean(text):
#     '''Make text lowercase, remove text in square brackets, remove links, remove punctuation
#     and remove words containing numbers.'''
#     stop_words = set(STOPWORDS)
#     words = nltk.word_tokenize(text)
#     # Use a regular function instead of lambda for multiline operations
#     def process_word(word):
#         word = str(word).lower()
#         word = re.sub('\[.*?\]', '', word)
#         word = re.sub('https?://\S+|www\.\S+', '', word)
#         word = re.sub('<.*?>+', '', word)
#         word = re.sub('\n', '', word)
#         word = re.sub('\w*\d\w*', '', word)
#         word = re.sub(r'@[A-Za-z0-9]+', '', word)
#         word = re.sub(r'#', '', word)
#         word = re.sub(r'RT[\s]+', '', word)
#         word = re.sub(r'[^\w]', ' ', word)
#         return word

#     filtered_words = [process_word(word) for word in words if word.lower() not in stop_words]
#     filtered_sentence = ' '.join(filtered_words)
#     return filtered_sentence

import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def clean(text):
    '''Make text lowercase, remove text in square brackets, remove links, remove punctuation,
    and remove words containing numbers and single letters (e.g., p, m, a, b, etc.).'''
    
    # Define stop words and punctuation
    stop_words = set(stopwords.words('english'))
    punctuation = string.punctuation

    # Tokenize the text
    words = word_tokenize(text)

    # Use a regular function instead of lambda for multiline operations
    def process_word(word):
        word = word.lower()
        word = re.sub('\[.*?\]', '', word)  # Remove text in square brackets
        word = re.sub('https?://\S+|www\.\S+', '', word)  # Remove links
        word = re.sub('<.*?>+', '', word)  # Remove HTML tags
        word = re.sub('\n', '', word)  # Remove newline characters
        word = re.sub(r'@[A-Za-z0-9]+', '', word)  # Remove mentions
        word = re.sub(r'#', '', word)  # Remove hashtags
        word = re.sub(r'RT[\s]+', '', word)  # Remove retweet indicators
        word = re.sub(f'[{punctuation}]', '', word)  # Remove punctuation
        word = re.sub(r'\b\w\b', '', word)  # Remove single letters

        return word

    # Filter and join the cleaned words
    filtered_words = [process_word(word) for word in words if word.lower() not in stop_words]
    filtered_sentence = ' '.join(filtered_words)
    return filtered_sentence


def getPolarity(text):
    return TextBlob(text).sentiment.polarity

def getAnalysis(score):
    if score < 0:
        return 0
    elif score==0:
        return 0
    else:
        return 1

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

In [14]:
tweets_biden = pd.read_csv('hashtag_joebiden.csv', lineterminator='\n', parse_dates=True)

In [15]:
tweets_biden['country'].replace({'United States':'United States of America'}, inplace=True)

In [16]:
tweets_biden_tmp = tweets_biden.sort_values('user_followers_count',ascending = False).drop_duplicates(['user_name'])[['tweet','country']]
tweets_biden_us = tweets_biden_tmp.dropna()
#.loc[tweets_biden_tmp.country == 'United States of America']

In [17]:
tweets_biden_us.count()

tweet      124624
country    124624
dtype: int64

In [18]:
tweets_biden_us.reset_index(inplace = True, drop = True)
tweets_biden_us['CleanTweet']=tweets_biden_us['tweet'].apply(clean)
tweets_biden_us['polarity']=tweets_biden_us['CleanTweet'].apply(getPolarity)
tweets_biden_us['sentiment']=tweets_biden_us['polarity'].apply(getAnalysis)
tweets_biden_us['language']=tweets_biden_us["CleanTweet"].apply(detect_language)
tweets_biden_us=tweets_biden_us.loc[tweets_biden_us.language == 'en']
tweets_biden_us.head()

C:\Users\SHUBHAM\AppData\Local\Temp\ipykernel_27260\2064647528.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SHUBHAM\AppData\Local\Temp\ipykernel_27260\2064647528.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SHUBHAM\AppData\Local\Temp\ipykernel_27260\2064647528.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

,tweet,country,CleanTweet,polarity,sentiment,language
1,President-elect #JoeBiden and Vice-President-e...,India,presidentelect joebiden vicepresidentelect k...,0.0000,0,en
2,U.S. debate commission says it will mute Democ...,China,us debate commission says mute democratic nomi...,0.0000,0,en
3,Signs of US becoming a failed state\n\n#JoeBid...,India,signs us becoming failed state joebiden may p...,-0.0375,0,en
8,Bye Donald #biden #kamala https://t.co/t44oNMmhyG,Brazil,bye donald biden kamala https tcot44onmmhyg,0.0000,0,en
12,Hollywood celebrities took to the streets to c...,India,hollywood celebrities took streets celebrate ...,0.8000,1,en


In [19]:
def lemmatizer(text):
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

tweets_biden_us['CleanTweetLemmatize'] =  tweets_biden_us.apply(lambda x: lemmatizer(x['CleanTweet']), axis=1)
tweets_biden_us['CleanTweetLemmatize'] = tweets_biden_us['CleanTweetLemmatize'].str.replace('-PRON-', '')

In [20]:
def clean_text(text):
    hash_tag_removed = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', text)
    hash_tag_removed = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text)

    removed_links = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text, flags=re.MULTILINE)
    cleaned = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', removed_links)
    cleaned = ''.join([i if ord(i) < 128 else ' ' for i in cleaned])
    
    cleaned = cleaned.strip()
    return cleaned

In [21]:
tweets_biden_us_copy = tweets_biden_us
tweets_biden_us['tweet']=tweets_biden_us['CleanTweetLemmatize'].apply(clean_text)

In [ ]:
# mpl.rcParams['figure.figsize']=(12.0,12.0)
# mpl.rcParams['font.size']=12
# mpl.rcParams['savefig.dpi']=100
# mpl.rcParams['figure.subplot.bottom']=.1
# # stopwords = set(STOPWORDS)

In [ ]:
# wordcloud = WordCloud(background_color='white',
#                       max_words=500,
#                       max_font_size=40,
#                       random_state=42
#                       ).generate(str(tweets_biden_us['CleanTweetLemmatize']))

In [ ]:
# print(wordcloud)
# fig = plt.figure(1)
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.show()

In [22]:
text_data = [text.split() for text in tweets_biden_us['tweet']]
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

In [23]:
lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=0, passes=10)
topics = lda_model.print_topics(num_words=20)

In [24]:
for topic in topics:
    print(topic)

(0, '0.012*"word" + 0.010*"bye" + 0.010*"bidenharristosaveamerica" + 0.010*"fire" + 0.009*"cry" + 0.007*"forget" + 0.006*"orange" + 0.006*"yourefire" + 0.005*"I" + 0.005*"electionresult" + 0.004*"dog" + 0.004*"hillaryclinton" + 0.004*"dumptrump2020" + 0.004*"makeamericagreatagain" + 0.004*"germany" + 0.003*"wonderful" + 0.003*"empathy" + 0.003*"virus" + 0.003*"trumpcrimefamily" + 0.003*"bitcoin"')
(1, '0.049*"biden" + 0.038*"vote" + 0.023*"trump" + 0.017*"https" + 0.012*"state" + 0.012*"win" + 0.010*"count" + 0.010*"lead" + 0.009*"election2020" + 0.008*"ballot" + 0.007*"georgia" + 0.007*"blue" + 0.007*"pennsylvania" + 0.007*"election" + 0.007*"michigan" + 0.007*"nevada" + 0.007*"poll" + 0.005*"elections2020" + 0.005*"call" + 0.004*"electionresults2020"')
(2, '0.072*"https" + 0.068*"joebiden" + 0.052*"biden" + 0.026*"trump" + 0.024*"president" + 0.022*"election2020" + 0.018*"kamalaharris" + 0.015*"election" + 0.014*"america" + 0.012*"congratulation" + 0.011*"bidenharris2020" + 0.011*"us

In [ ]:
# from collections import Counter
# word_freq = Counter()
# for text in text_data:
#     word_freq.update(text)

In [28]:
# for i, topic in lda_model.show_topics(num_words=20, formatted=False):
#     topic_words = [word for word, _ in topic]
#     topic_wordcloud = WordCloud(
#         background_color='white',
#         max_words=200,
#         max_font_size=40,
#         random_state=42,
#     ).generate_from_frequencies({word: word_freq[word] for word in topic_words})

In [29]:
# plt.figure(figsize=(8, 6), facecolor=None)
# plt.imshow(topic_wordcloud)
# plt.axis("off")
# plt.title(f"Topic {i + 1}")
# plt.show()

In [25]:
!pip install daal==2021.4.0 --quiet
!pip install numpy==1.18.5 --quiet
#!pip install --upgrade pip

ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [90 lines of output]
  Running from numpy source directory.
  <string>:461: UserWarning: Unrecognized setuptools command, proceeding with generating Cython sources and expanding templates
  C:\Users\SHUBHAM\AppData\Local\Temp\pip-install-pqib0r2i\numpy_1606744b12644bea9eafb1c4440188cf\tools\cythonize.py:75: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
    required_version = LooseVersion('0.29.14')
  C:\Users\SHUBHAM\AppData\Local\Temp\pip-install-pqib0r2i\numpy_1606744b12644bea9eafb1c4440188cf\tools\cythonize.py:77: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
    if LooseVersion(cython_version

In [26]:
!pip install pyLDAvis --quiet

In [27]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim

In [28]:
pyLDAvis.enable_notebook()

In [29]:
vis = pyLDAvis.gensim.prepare(topic_model=lda_model,corpus=corpus,dictionary=dictionary)

In [30]:
pyLDAvis.save_html(vis, 'lda_assignment3.html')

In [31]:
pyLDAvis.display(vis)